[Previously](http://www.kaggle.com/rowena1/my-first-kernel), through undersampling the majority class, I was able to get a Random Forest classifier that outperforms a no-skill dummy classifier -- area under precision recall curve was 0.4816 compared to 0.2538 in the no-skill case.  The aim of this exercise is to improve upon the previous classifier.

Here, instead of undersampling the majority class, I used Synthetic Minority Over-sampling Technique. This raised the area under the precision-recall curve from 0.4816 to 0.5445.  Then, by tweaking the hyperparameters improved model performance further, bringing the area under the precision-recall curve to 0.8163.  Also, this was achieved with a test set that preserved the imbalance between the majority and minority classes to better represent reality.  However, since PCA was applied to the original dataset to protect customer privacy, there would have been some information leaked to the test set.

Searching through various combinations of hyperparameters is time consuming.  It is more efficient to first launch an interim model using default hyperparametric settings and an edited set of predictors.  Then, search for an optimized model and release that.  Using a more parsimonious set of predictors also aids interpretability of the decision paths. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['creditcard.csv']


In [2]:
#Get tools
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import time
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, average_precision_score, auc, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [3]:
df=pd.read_csv('../input/creditcard.csv')
print(df.head())

   Time        V1        V2        V3  ...         V27       V28  Amount  Class
0   0.0 -1.359807 -0.072781  2.536347  ...    0.133558 -0.021053  149.62      0
1   0.0  1.191857  0.266151  0.166480  ...   -0.008983  0.014724    2.69      0
2   1.0 -1.358354 -1.340163  1.773209  ...   -0.055353 -0.059752  378.66      0
3   1.0 -0.966272 -0.185226  1.792993  ...    0.062723  0.061458  123.50      0
4   2.0 -1.158233  0.877737  1.548718  ...    0.219422  0.215153   69.99      0

[5 rows x 31 columns]


In [4]:
#From previous exercise, unsupporting variables and variables with collinearity were identified
#Get rid of those for a simpler set of predictors
df.drop(['Amount','Time'],axis=1,inplace=True)
df.drop(['V8','V13','V23','V26','V27','V28'],axis=1,inplace=True)
df.drop(['V2','V3','V5','V7','V9','V11','V15','V19','V20','V21','V22','V24','V25'],axis=1,inplace=True)
print(df.head())

         V1        V4        V6  ...         V17       V18  Class
0 -1.359807  1.378155  0.462388  ...    0.207971  0.025791      0
1  1.191857  0.448154 -0.082361  ...   -0.114805 -0.183361      0
2 -1.358354  0.379780  1.800499  ...    1.109969 -0.121359      0
3 -0.966272 -0.863291  1.247203  ...   -0.684093  1.965775      0
4 -1.158233  0.403034  0.095921  ...   -0.237033 -0.038195      0

[5 rows x 10 columns]


In [5]:
#As in previous exercise, imbalanced class ratio warrants
#(1)training set with balanced class representation; and 
#(2)an unseen test set with a similar distribution of minority to majority class (fraud ~ 0.17%)

#Instead of undersampling the majority class, here I use Synthetic Minority Over-sampling Technique (SMOTE)

#First, split into train and test sets to ensure no information leaked to test set

X,y=df.iloc[:,:-1],df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

#Check majority vs. minority class distribution in train and test sets

print('Fraudulent share, train set (before SMOTE): {0:.2%}'.format(sum(y_train==1)/len(y_train)))
print('Fraudulent share, test set: {0:.2%}'.format(sum(y_test==1)/len(y_test))) 

Fraudulent share, train set (before SMOTE): 0.17%
Fraudulent share, test set: 0.17%


In [6]:
#Apply SMOTE to train set
sm=SMOTE(random_state=22)
X_resampled, y_resampled=sm.fit_sample(X_train,y_train)

#Check majority vs. minority class distribution in train set after resampling

print('Fraudulent share, train set (after SMOTE): {0:.2%}'.format(sum(y_resampled==1)/len(y_resampled)))

Fraudulent share, train set (after SMOTE): 50.00%


In [7]:
#Fit Random Forest Classifier used previously to data
#Predict and evaluate effect of SMOTE

RFC_mod=RandomForestClassifier(max_depth=4,random_state=22)

RFC_mod.fit(X_resampled,y_resampled)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=22, verbose=0,
                       warm_start=False)

In [8]:
#Predict and check AUPRC as well as time the prediction

t0=time.time()
y_pred_RFC=RFC_mod.predict(X_test)
t1=time.time()

print('Predicting with Random Forest Classifier model took: {0:.4f} seconds'.format(t1-t0))

labels=['No Fraud','Fraud']
#Calculate precision recall curve and area under curve
precision_RFC,recall_RFC,threshold_RFC=precision_recall_curve(y_test,y_pred_RFC)
auprc_RFC=auc(recall_RFC,precision_RFC)
print()
print('Area under precision recall curve, Random Forest Classifier model: {0:.4f}'.format(auprc_RFC))
print()
print(classification_report(y_test,y_pred_RFC,target_names=labels))
print()
print(confusion_matrix(y_test,y_pred_RFC))

Predicting with Random Forest Classifier model took: 0.0468 seconds

Area under precision recall curve, Random Forest Classifier model: 0.5445

              precision    recall  f1-score   support

    No Fraud       1.00      0.99      1.00     85295
       Fraud       0.20      0.89      0.32       148

    accuracy                           0.99     85443
   macro avg       0.60      0.94      0.66     85443
weighted avg       1.00      0.99      1.00     85443


[[84757   538]
 [   16   132]]


In [9]:
#Using SMOTE instead of undersampling the majority the class improved model performance
#Area under precision recall curve increased to 0.5445 from 0.4824
#Prediction still took under one second - YAY!

#Let's see if changing hyperparameters of Random Forest Classifier can improve model performance further
#Here, I choose to tweak the 
#(1)max_depth (maximum depth of each tree), and 
#(2)n_estimators (number of trees).

# Maximum depth of each tree
max_depth=[None,5,10]

# Number of trees
n_estimators=[10,12,14,16]

#create grid
grid={
    'max_depth': max_depth,
    'n_estimators': n_estimators
}

#Random search of parameters
RFC_search=GridSearchCV(estimator=RandomForestClassifier(random_state=22), param_grid=grid,cv=3,scoring='f1',verbose=True)

#Fit model
RFC_search.fit(X_resampled, y_resampled)

#print results
print(RFC_search.best_score_)
print(RFC_search.best_params_)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  5.7min finished


0.9995956187669541
{'max_depth': None, 'n_estimators': 16}


In [10]:
#Wow! That took a long time for hardly an exhaustive search!

#The best max_depth and n_estimators identified are:
#'None' and '16', respectively.
#Refit the random forest classifier with these hyperparameters and check performance

RFC_mod=RandomForestClassifier(random_state=22,max_depth=None,n_estimators=16)
RFC_mod.fit(X_resampled,y_resampled)

t0=time.time()
y_pred_RFC=RFC_mod.predict(X_test)
t1=time.time()

print('Predicting with Random Forest Classifier model took: {0:.4f} seconds'.format(t1-t0))

#Calculate precision recall curve and area under curve
precision_RFC,recall_RFC,threshold_RFC=precision_recall_curve(y_test,y_pred_RFC)
auprc_RFC=auc(recall_RFC,precision_RFC)
print()
print('Area under precision recall curve, Random Forest Classifier model: {0:.4f}'.format(auprc_RFC))
print()
print(classification_report(y_test,y_pred_RFC,target_names=labels))
print()
print(confusion_matrix(y_test,y_pred_RFC))

Predicting with Random Forest Classifier model took: 0.1233 seconds

Area under precision recall curve, Random Forest Classifier model: 0.8163

              precision    recall  f1-score   support

    No Fraud       1.00      1.00      1.00     85295
       Fraud       0.81      0.82      0.82       148

    accuracy                           1.00     85443
   macro avg       0.90      0.91      0.91     85443
weighted avg       1.00      1.00      1.00     85443


[[85266    29]
 [   26   122]]


In [11]:
#Great! The area under the precision recall curve rose to 0.8163
#and the number of false positives and false negatives limited.

#Since the Random Forest Classifier is a 'bagging' algorithm, 
#wouldn't increasing the number of trees improve performance?
#Right now, the default number of trees is 10; but this will be changed to 100 soon.
#Let's fit this soon-to-be 'default' model to our training set and compare the performance.

RFC_mod=RandomForestClassifier(random_state=22,max_depth=None,n_estimators=100)
RFC_mod.fit(X_resampled,y_resampled)

t0=time.time()
y_pred_RFC=RFC_mod.predict(X_test)
t1=time.time()

print('Predicting with Random Forest Classifier model took: {0:.4f} seconds'.format(t1-t0))

#Calculate precision recall curve and area under curve
precision_RFC,recall_RFC,threshold_RFC=precision_recall_curve(y_test,y_pred_RFC)
auprc_RFC=auc(recall_RFC,precision_RFC)
print()
print('Area under precision recall curve, Random Forest Classifier model: {0:.4f}'.format(auprc_RFC))
print()
print(classification_report(y_test,y_pred_RFC,target_names=labels))
print()
print(confusion_matrix(y_test,y_pred_RFC))

Predicting with Random Forest Classifier model took: 0.7475 seconds

Area under precision recall curve, Random Forest Classifier model: 0.8042

              precision    recall  f1-score   support

    No Fraud       1.00      1.00      1.00     85295
       Fraud       0.77      0.84      0.80       148

    accuracy                           1.00     85443
   macro avg       0.88      0.92      0.90     85443
weighted avg       1.00      1.00      1.00     85443


[[85258    37]
 [   24   124]]


Not as good as the classifier using n_estimators=16 but, still, comparable performance.  It seems more efficient to spend time identifying supportive / correlated features before fitting a model than to depend on an exhaustive search for the best hyperparameters.  Even though the Random Forest Classifier was developed to minimize overfitting, it is not immune to it.    